In [1]:
# script to save the rankings for the mechanism
import os
import sys
import copy
import pickle
import pandas as pd

import numpy as np
import rmgpy.chemkin
import cantera as ct

import matplotlib.pyplot as plt
%matplotlib inline

## Load the model for reaction and species descriptions

In [2]:
basedir = '/work/westgroup/harris.se/autoscience/reaction_calculator/delay_uncertainty/base_rmg_1week'

cantera_file = os.path.join(basedir, 'chem_annotated.cti')
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')

species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport, use_chemkin_names=True)

gas = ct.Solution(cantera_file)
perturbed_cti_path = os.path.join(basedir, 'perturbed.cti')
perturbed_gas = ct.Solution(perturbed_cti_path)

# This cti -> rmg converter dictionary can be made using rmg_tools/ct2rmg_dict.py
with open(os.path.join(basedir, 'ct2rmg_rxn.pickle'), 'rb') as handle:
    ct2rmg_rxn = pickle.load(handle)

print(f'{len(species_list)} species loaded')
print(f'{len(reaction_list)} reactions loaded')

For species Ar, discontinuity in h/RT detected at Tmid = 4762.74
	Value computed using low-temperature polynomial:  2.327935178063383
	Value computed using high-temperature polynomial: 2.320328165487312

For species He, discontinuity in h/RT detected at Tmid = 4762.74
	Value computed using low-temperature polynomial:  2.327935178063383
	Value computed using high-temperature polynomial: 2.320328165487312

For species butane(1), discontinuity in h/RT detected at Tmid = 1389.0
	Value computed using low-temperature polynomial:  6.672352745666894
	Value computed using high-temperature polynomial: 6.314788936675075

For species CH(3), discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  81.27192833333334
	Value computed using high-temperature polynomial: 81.30542726833333

For species C2H(4), discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  79.04515833333333
	Value computed using high-temperature 

130 species loaded
2488 reactions loaded


In [3]:
N = len(gas.species())
M = len(gas.reactions())

## Gather Uncertainty Rankings

In [4]:
# rxn_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/reaction_uncertainty.npy'
# sp_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/species_uncertainty.npy'

rxn_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/gao_reaction_uncertainty.npy'
sp_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/gao_species_uncertainty.npy'

rmg_rxn_uncertainty = np.load(rxn_uncertainty_file)
rmg_sp_uncertainty = np.load(sp_uncertainty_file)

assert len(rmg_rxn_uncertainty) == len(reaction_list)
assert len(rmg_sp_uncertainty) == len(species_list)


rxn_uncertainty = np.zeros(len(gas.reactions()))
for ct_index in range(len(rxn_uncertainty)):
    rxn_uncertainty[ct_index] = rmg_rxn_uncertainty[ct2rmg_rxn[ct_index]]

# Cantera species should be in same rmg order, but this makes sure for us
for i in range(len(species_list)):
    assert str(species_list[i]) == gas.species_names[i]

sp_uncertainty = rmg_sp_uncertainty


In [5]:
# Display the most uncertain parameters

In [6]:
# Calculate Improvement Score
DFT_error = 3.0
# improvement_score = np.abs(reaction_sensitivities) * (uncorrelated_uncertainties - DFT_error)
# improvement_score_order = [x for _,x in sorted(zip(improvement_score, reaction_indices))][::-1]
reaction_indices = np.arange(0, len(gas.reactions()))
reaction_uncertainty_order = [x for _,x in sorted(zip(rxn_uncertainty, reaction_indices))][::-1]

In [7]:
print('Top Uncertain Reactions')
print('i\tDelta\tReaction\tSensitivity\tImprovement Score')
for i in range(0, 50):
    ct_index = reaction_uncertainty_order[i]
    print(ct_index, '\t', np.round(rxn_uncertainty[ct_index], 3),
          '\t', gas.reactions()[ct_index], 
          '\t', reaction_list[ct2rmg_rxn[ct_index]].family)
#           '\t', f'{reaction_sensitivities[ct_index]:.3e}',
#           '\t', f'{improvement_score[ct_index]:.3e}')
    

Top Uncertain Reactions
i	Delta	Reaction	Sensitivity	Improvement Score
2174 	 28.541 	 C3H6(688) + CH3(18) <=> C3H5-A(94) + CH4(10) 	 Disproportionation
2168 	 28.541 	 C3H6(688) + CH2(23) <=> C3H5-A(94) + CH3(18) 	 Disproportionation
1251 	 28.541 	 C3H5O(129) + H(14) <=> C3H4O(74) + H2(13) 	 Disproportionation
1248 	 28.541 	 C3H5O(129) + CH3(18) <=> C3H4O(74) + CH4(10) 	 Disproportionation
1242 	 28.541 	 C3H5O(129) + CH2(23) <=> C3H4O(74) + CH3(18) 	 Disproportionation
1001 	 28.541 	 C2H5O(49) + CH2(23) <=> CH3(18) + CH3CHO(35) 	 Disproportionation
1000 	 28.541 	 C2H5O(49) + CH3(18) <=> CH3CHO(35) + CH4(10) 	 Disproportionation
958 	 28.541 	 CH2(23) + S(777) <=> CH3(18) + S(252) 	 Disproportionation
955 	 28.541 	 H(14) + S(777) <=> H2(13) + S(252) 	 Disproportionation
953 	 28.541 	 CH3(18) + S(777) <=> CH4(10) + S(252) 	 Disproportionation
790 	 28.541 	 C4H8(748) + CH3(18) <=> C4H7(190) + CH4(10) 	 Disproportionation
780 	 28.541 	 C4H8(748) + CH2(23) <=> C4H7(190) + CH3(18) 

## Gather Sensitivity Rankings

In [31]:
# load the giant base delays matrix
base_delay_file = os.path.join(basedir, 'total_base_delays.npy')
base_delays = np.load(base_delay_file)

# Load the giant delays matrix
total_delay_file = os.path.join(basedir, 'total_perturbed_mech_delays.npy')
total_delays = np.load(total_delay_file)

assert total_delays.shape[1] == len(base_delays)

total_base_delays = np.repeat(np.matrix(base_delays), total_delays.shape[0], axis=0)
total_base_delays[total_base_delays == 0] = np.nan
assert total_base_delays.shape == total_delays.shape

total_delays[total_delays == 0] = np.nan


In [32]:
d_ln_tau = np.log(total_delays) - np.log(total_base_delays)

In [47]:
avg_d_ln_tau = np.nanmean(d_ln_tau, axis = 1)
avg_d_ln_tau[np.isnan(avg_d_ln_tau)] = -np.inf

/work/westgroup/harris.se/tst_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


In [48]:
parameter_indices = np.arange(0, N + M)
reaction_sensitivity_order = [x for _, x in sorted(zip(avg_d_ln_tau, parameter_indices))][::-1]

In [50]:
print('Top Sensitive Parameters')
print('i\tDelta\tReaction\tSensitivity\tImprovement Score')
for i in range(0, 50):
    ct_index = reaction_sensitivity_order[i]
    
    if ct_index < N:
        print(ct_index, '\t', np.round(avg_d_ln_tau[ct_index,0], 9),
              '\t', gas.species()[ct_index], )
    else:
        print(ct_index, '\t', np.round(avg_d_ln_tau[ct_index,0], 9),
              '\t', gas.reactions()[ct_index - N])

Top Sensitive Parameters
i	Delta	Reaction	Sensitivity	Improvement Score
97 	 0.762681699 	 <Species S(787)>
66 	 0.563358541 	 <Species S(186)>
65 	 0.320233344 	 <Species S(184)>
398 	 0.048299511 	 OH(15) + butane(1) <=> H2O(8) + SC4H9(183)
5 	 0.042960903 	 <Species O2(2)>
4 	 0.038979979 	 <Species butane(1)>
190 	 0.030488837 	 2 HO2(16) <=> H2O2(17) + O2(2)
150 	 0.030488821 	 2 HO2(16) <=> H2O2(17) + O2(2)
430 	 0.028937494 	 S(186) <=> C4H8(189) + HO2(16)
55 	 0.024694828 	 <Species C3H5-A(94)>
67 	 0.021250122 	 <Species S(187)>
468 	 0.019870428 	 S(184) <=> C4H8(188) + HO2(16)
469 	 0.019050385 	 S(186) <=> C4H8(188) + HO2(16)
49 	 0.018676758 	 <Species C3H5O3(72)>
70 	 0.016875988 	 <Species C4H7(190)>
603 	 0.014679425 	 S(219) <=> C4H8O(214) + OH(15)
13 	 0.012118072 	 <Species CH4(10)>
25 	 0.011777576 	 <Species C2H3(22)>
45 	 0.011537143 	 <Species C2H3OH(58)>
17 	 0.011225498 	 <Species H(14)>
18 	 0.00969352 	 <Species OH(15)>
102 	 0.008224239 	 <Species S(928)>
29

In [ ]:
phi_dicts = []

for table_index in range(1, 13):
    
    # Load the experimental conditions
    ignition_delay_data = '/work/westgroup/harris.se/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
    # ignition_delay_data = '/home/moon/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
    df_exp = pd.read_csv(ignition_delay_data)
    table_exp = df_exp[df_exp['Table'] == table_index]
    # Define Initial conditions using experimental data
    tau_exp = table_exp['time (ms)'].values.astype(float)  # ignition delay
    T7 = table_exp['T_C'].values  # Temperatures
    P7 = table_exp['nominal pressure(atm)'].values * ct.one_atm  # pressures in atm
    phi7 = table_exp['phi'].values  # equivalence ratios
    # list of starting conditions
    # Mixture compositions taken from table 2 of
    # https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
    concentrations = []
    # for phi = 1
    x_diluent = 0.7649
    conc_dict = {
        'O2(2)': 0.2038,
        'butane(1)': 0.03135
    }

    
    x_N2 = table_exp['%N2'].values[0] / 100.0 * x_diluent
    x_Ar = table_exp['%Ar'].values[0] / 100.0 * x_diluent
    x_CO2 = table_exp['%CO2'].values[0] / 100.0 * x_diluent
    conc_dict['N2'] = x_N2
    conc_dict['Ar'] = x_Ar
    conc_dict['CO2(7)'] = x_CO2
    
    phi_dicts.append(conc_dict)
        
        

In [ ]:
# There are 12 * K different simulation settings. We need each parameter estimate at each setting
# Create a matrix with temperatures and one with pressures
T = np.linspace(663, 1077, 51)
table_temperatures = np.repeat(np.matrix(T), 12, axis=1)
temperatures = np.repeat(table_temperatures, total_delays.shape[0], axis=0)

pressures = np.zeros(temperatures.shape)
for i in range(pressures.shape[1]):
    if int(i / 51) in [0, 3, 6, 9]:
        pressures[:, i] = 10.0 * 101325.0
    elif int(i / 51) in [1, 4, 7, 10]:
        pressures[:, i] = 20.0 * 101325.0
    elif int(i / 51) in [2, 5, 8, 11]:
        pressures[:, i] = 30.0 * 101325.0

        
phis = np.zeros(temperatures.shape)
for i in range(phis.shape[1]):
    if int(i / 153) == 0:
        phis[:, i] = 0.3
    elif int(i / 153) == 1:
        phis[:, i] = 0.5
    elif int(i / 153) == 2:
        phis[:, i] = 1.0
    elif int(i / 153) == 3:
        phis[:, i] = 2.0

# P = np.array([10, 20, 30, 10, 20, 30, 10, 20, 30, 10, 20, 30]) * 101325.0
# pressures = np.repeat(np.matrix(P), total_delays.shape[0], axis=0)

In [ ]:
G_base = np.zeros((N, total_delays.shape[1]))
G_perturbed = np.zeros((N, total_delays.shape[1]))


# get base G values

mod_gas = ct.Solution(cantera_file)
for j in range(N):
    for i in range(temperatures.shape[1]):
    
        T = temperatures[0, i]
        gas.TPX = T, pressures[0, i], phi_dicts[int(i / 51)]
        G_base[j, i] = gas.species()[j].thermo.h(T) - T * gas.species()[j].thermo.s(T)


In [ ]:
# Get perturned G values

mod_gas = ct.Solution(cantera_file)
for j in range(N):
#     print(j)
    # change just the one reaction
    mod_gas.modify_species(j, perturbed_gas.species()[j])
    
    for i in range(temperatures.shape[1]):
    
        T = temperatures[0, i]
        mod_gas.TPX = T, pressures[0, i], phi_dicts[int(i / 51)]
        G_perturbed[j, i] = mod_gas.species()[j].thermo.h(T) - T * mod_gas.species()[j].thermo.s(T)

    mod_gas.modify_species(j, gas.species()[j])

In [ ]:
# G has units Enthalpy [J/kg or J/kmol] it's J / kmol
delta_G = G_perturbed - G_base
delta_G_kcal_mol = delta_G / 4.184 / 1000.0 / 1000.0  # needs to be kcal/mol to match Gao paper

### Now do reaction rate

In [ ]:
# except we know that by definition, this is 0.1

In [ ]:
delta_k = 0.1 * np.ones((M, total_delays.shape[1]))

In [ ]:
# concatenate into a big delta matrix
delta = np.concatenate((delta_G_kcal_mol, delta_k), axis=0)

In [ ]:
# first derivative is change in delay / change in G
first_derivative = np.divide(d_ln_tau, delta)

In [ ]:
# multiply uncertainty times change in sensitivity
total_uncertainty_array = np.concatenate((sp_uncertainty, rxn_uncertainty), axis=0)
total_uncertainty_mat = np.repeat(np.transpose(np.matrix(total_uncertainty_array)), first_derivative.shape[1], axis=1)


In [ ]:
delta_uncertainty = total_uncertainty_mat - 3.0

In [ ]:
improvement_score = np.multiply(np.abs(first_derivative), delta_uncertainty)

In [ ]:
improvement_score

In [ ]:
avg_improvement_score = np.nanmean(improvement_score, axis=1)

In [ ]:
index = 7 * 51 + 7
my_improvement_score = np.multiply(np.abs(first_derivative[:, index]), delta_uncertainty[:, index])


param_indices = np.arange(0, len(my_improvement_score))
order = [x for _, x in sorted(zip(my_improvement_score, param_indices))][::-1]

In [ ]:
my_improvement_score[13]

In [ ]:
my_improvement_score[14]

In [ ]:
order[14]

In [ ]:
order[13]

In [ ]:


for i in range(0, 50):
    ct_index = order[i]
    if ct_index < N:
        print(gas.species()[ct_index])
    else:
        print(i, gas.reactions()[ct_index - N], my_improvement_score[ct_index])

In [ ]:
# rank by average improvement score

avg_improvement_score


param_indices = np.arange(0, len(avg_improvement_score))

X = ["a", "b", "c", "d", "e", "f", "g", "h", "i"]
Y = [ 0,   1,   1,    0,   1,   2,   2,   0,   1]

order = [x for _, x in sorted(zip(avg_improvement_score, param_indices))][::-1]


In [ ]:
avg_improvement_score[2652]

In [ ]:
order[-1]

In [ ]:
avg_improvement_score[4]

In [ ]:
for i in range(0, 50):
    ct_index = order[i]
    if ct_index < N:
        print(gas.species()[ct_index])
    else:
        print(gas.reactions()[ct_index - N])
    

In [ ]:


for j in range(N + M):
    if np.isnan(fd[j]):
        if j < N:
            print(gas.species()[j])
        else:
            print(type(gas.reactions()[j - N]))